# TP 2 Multitâches
# Exercice 1 - Pi Monte Carlo

## Introduction

Cet exercice nécessite le package **Numba**. 

Ce TP a pour but de paralléliser l'algorithme de pi par Monte Carlo en utilisant **multithreading** et **multiprocessing**.

In [9]:
import numpy as np
from numba import njit
import time
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

La méthode `pick` tire `n` coups dans le carré $[-1,1] \times [-1,1]$ et retourne le nombre de coups tirés dans le disque inscrit au carré.

In [2]:
@njit
def pick(n):
    '''
    input : n nombre de tirage dans le carre [-1, 1]
    output: count_inside nombre de coups tires dans le disque inscrit au carre
    '''
    count_inside = 0
    for i in range(n):
        x, y = np.random.random(2) * 2 - 1
        if x**2 + y**2 <= 1: count_inside += 1
    return count_inside

La méthode pi_mc appel la méthode pick sur la valeur n et retourne la valeur approchée $\pi$ par la formule $4 \times p_C/p_T$ où $p_C$ désigne le nombre de coups dans le disque et $p_T$ le nombre de coups total.

In [8]:
def pi_mc(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    tic = time.time()
    api = 4 * pick(n) / n
    toc = time.time()
    print(f'For {n} picks and 1 process, the approximation of pi is {api} compute in {toc - tic} seconds')
    return api

if (__name__ == "__main__"):
    n = 10_000
    pi_mc(n)

    n = 10_000_000
    pi_mc(n)

For 10000 picks and 1 process, the approximation of pi is 3.1424 compute in 0.004734992980957031 seconds
For 10000000 picks and 1 process, the approximation of pi is 3.1411464 compute in 0.8369674682617188 seconds



## 1 - Parallélisation avec multiprocessing

1.a) Sur la base de `pi_mc` créer une fonction `pi_mc_mp` qui répartit le travail entre plusieurs processus à l'aide de `multiprocessing` comme vu en cours.


In [4]:
def pi_mc_mp(n,p):
    tic = time.time()
    chunk = [n//p for i in range(p)]
    p_exe = ProcessPoolExecutor()
    result = [res for res in p_exe.map(pick,chunk)]
    count = sum(result)
    pi = 4 * count / n
    toc = time.time()
    print(f'For {n} picks and {p} processes, the approximation of pi is {pi} compute in {toc - tic} seconds')


if (__name__ == "__main__"):
    n = 10000 #int(sys.argv[1])
    p = 10
    pi_mc_mp(n,p)


For 10000 picks and 10 processes, the approximation of pi is 3.1764 compute in 0.2568488121032715 seconds


1.b) Mesurer les temps de restitution en variant le nombre de tir et le nombre de processus.

In [5]:
values_n = [10_000, 10_000, 10_000, 10_000, 10_000_000, 10_000_000, 10_000_000, 10_000_000]
values_p = [10, 20, 50, 100, 10, 50, 100, 1000]

for i in range(len(values_n)):
    pi_mc_mp(values_n[i], values_p[i])

For 10000 picks and 10 processes, the approximation of pi is 3.148 compute in 0.042574405670166016 seconds
For 10000 picks and 20 processes, the approximation of pi is 3.118 compute in 0.0457758903503418 seconds
For 10000 picks and 50 processes, the approximation of pi is 3.146 compute in 0.05580639839172363 seconds
For 10000 picks and 100 processes, the approximation of pi is 3.1336 compute in 0.08130240440368652 seconds
For 10000000 picks and 10 processes, the approximation of pi is 3.1419704 compute in 0.21980714797973633 seconds
For 10000000 picks and 50 processes, the approximation of pi is 3.14177 compute in 0.19802546501159668 seconds
For 10000000 picks and 100 processes, the approximation of pi is 3.1423064 compute in 0.2676541805267334 seconds
For 10000000 picks and 1000 processes, the approximation of pi is 3.1416508 compute in 0.5803806781768799 seconds


Nous pouvons constater que l’augmentation du nombre de processus a tendance à entraîner une légère augmentation du temps d’exécution.

Diviser le travail en plusieurs processus accélère le processus par rapport à la fonction pi_mc() dans le cas de 10_000_000 picks. Cependant, dans le cas de seulement 10 000 sélections, le multitraitement est moins efficace, probablement en raison de la nécessité de transférer des informations entre les processus et d'effectuer d'autres étapes de division de blocs.


## 2 - Parallélisation avec multithreading

2.a) Sur la base de `pi_mc_mp` créer une fonction `pi_mc_mt` qui répartit le travail entre plusieurs threads à l'aide de `multithreading` comme vu en cours.


In [6]:
def pi_mc_mt(n,p):
    tic = time.time()
    chunk = [n//p for i in range(p)]
    t_exe = ThreadPoolExecutor()
    result = [res for res in t_exe.map(pick,chunk)]
    count = sum(result)
    pi = 4 * count / n
    toc = time.time()
    print(f'For {n} picks and {p} processes, the approximation of pi is {pi} compute in {toc - tic} seconds')


if (__name__ == "__main__"):
    n = 10000 #int(sys.argv[1])
    p = 10
    pi_mc_mt(n,p)

For 10000 picks and 10 processes, the approximation of pi is 3.142 compute in 0.0029447078704833984 seconds


2.b) Mesurer les temps de restitution en variant le nombre de tir et le nombre de processus. Comparer avec la méthode précédente.

In [7]:
for i in range(len(values_n)):
    pi_mc_mt(values_n[i], values_p[i])

For 10000 picks and 10 processes, the approximation of pi is 3.1536 compute in 0.002092123031616211 seconds
For 10000 picks and 20 processes, the approximation of pi is 3.1456 compute in 0.0030694007873535156 seconds
For 10000 picks and 50 processes, the approximation of pi is 3.1284 compute in 0.0050199031829833984 seconds
For 10000 picks and 100 processes, the approximation of pi is 3.1404 compute in 0.0035648345947265625 seconds
For 10000000 picks and 10 processes, the approximation of pi is 3.14127 compute in 0.7681381702423096 seconds
For 10000000 picks and 50 processes, the approximation of pi is 3.1420716 compute in 0.7781856060028076 seconds
For 10000000 picks and 100 processes, the approximation of pi is 3.1417544 compute in 0.7970001697540283 seconds
For 10000000 picks and 1000 processes, the approximation of pi is 3.1421096 compute in 0.8204166889190674 seconds


En divisant le travail en différents threads, le temps d'exécution est considérablement inférieur à celui de l'appel de fonction pi_mc() habituel. De plus, l'augmentation du nombre de sélections et de processus continue d'augmenter le temps d'exécution de la fonction.

Il convient de mentionner que pour que la parallélisation soit possible, il faut que :
- l'algorithme est parallélisable
- Le système opérationnel l'accepte
- La langue est l'accepte
- Il y a suffisamment de ressources matérielles

Les trois premières conditions sont toujours remplies pour cet exercice. Par conséquent, comme l’augmentation du nombre de threads entraîne un coût, une augmentation importante du nombre de processus peut atteindre la limite des ressources matérielles, ce qui peut avoir un impact sur ces résultats.